# Prediction based on CloudPred

In [1]:
import pickle
import scipy
import scipy.io
import os
import numpy as np
import scanpy as sc
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import sklearn.model_selection as sks
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from matplotlib.pyplot import rc_context
import sklearn.metrics as skm
import collections
from sklearn.model_selection import KFold
import torch
import traceback
import random
import pathlib
import sklearn.mixture
import math
from lib_cloudpred import *

In [2]:
def get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers):
            df = markers.loc[markers["cluster"].isin(set_clusters),:]
            feat_tab = df.groupby('cluster')
            df2= feat_tab.apply(lambda x: x.sort_values(["avg_log2FC"], ascending=False)).reset_index(drop=True)
            feat=df2.groupby('cluster').head(n)
            idx_te= np.where(adata1_c.isin (feat.gene.values))[0] 
            idx_tr= np.where(adata_c.isin (adata1_c[idx_te]))[0]            

#             markers = ['HLA-DRA','HLA-DRB1','LYZ','CST3','TYROBP','AP1S2','CSTA','FCN1','MS4A6A','LST1','CYBB','CTSS','DUSP6','IL1B','SGK1','KLF4','CLEC7A','ATP2B1-AS1','MARCKS','SAT1','MYADM','IFI27','IFITM3','ISG15','APOBEC3A','IFI6','TNFSF10','MT2A','MX1','IFIT3','MNDA','S100A12','S100A9','S100A8','MAFB','VCAN','PLBD1','CXCL8','RNASE2','FCGR3A','MS4A7','CDKN1C','AIF1','COTL1','FCER1G','C1QA','RHOC','FCGR3B','IFITM2','NAMPT','G0S2','PROK2','CMTM2','BASP1','BCL2A1','SLC25A37','DEFA3','LTF','LCN2','CAMP','RETN','DEFA4','CD24','PGLYRP1','OLFM4']
#             idx_tr= np.where(adata_c.isin (markers))[0]
#             idx_te= np.where(adata1_c.isin (markers))[0]

            Xtrain = list(map(lambda x: (x[0][:,idx_tr], *x[1:]), Xtrain))
            Xtest = list(map(lambda x: (x[0][:,idx_te], *x[1:]), Xtest))
            Xtrain = list(map(lambda x: (x[0].todense(), *x[1:]), Xtrain))
            Xtest  = list(map(lambda x: (x[0].todense(), *x[1:]), Xtest))    
            return Xtrain,Xtest

In [3]:
def cloud_pred(Xtrain, Xtest, seed, set_centers):
    
            best_model = None
            best_score = float("inf")
            print("start cloudpred")
            print(seed)
            for centers in set_centers:
                model, res = train(Xtrain, [], centers, regression=False, seed=seed)
                print("best score for center"+ str(res["loss"])+"_"+str( centers))
                print(res)
                if res["loss"] < best_score:
                    best_model = model
                    best_score = res["loss"]
                    best_centers = centers
                    res_ = res
            with open(path+'/cloudpred_model'+label+"_"+str(seed)+"_"+str(best_centers)+"_"+str(n), 'wb') as fp:
                 pickle.dump(best_model, fp)         
            print("##Validation")
            print(res_)
            res = eval(best_model, Xtest, regression=False,path=path+"/"+label+"/"+str(seed)+"_"+str(n)+"_")
            print("##Test")
            print(res)


## 1.Load data (all cells, Monocytes, Monocytes+neutrophils, Neutrophils)

In [4]:
data = collections.defaultdict(list)
path= "../../2cohorts/chr2chr1/"
files= ['Xtest','Xall','Ctest','Call','state']
ext= ['','_mono','_mono_neu','_neu']
for fl in files:
    for ex in ext:
        car= fl+ex
        with open(path+fl+ex+".pkl", "rb") as f:
            buf = pickle.load(f)
        if fl ==  'Xall':
            buf=np.concatenate([buf[0],buf[1]])
        data[car]=buf

## Load marker genes

In [5]:
markers= pd.read_csv('../scripts/marker_cohort2')
markers["avg_log2FC"] = np.abs(markers["avg_log2FC"])

## 2.Prediction based on Monocytes 

### 2.1.Using top 50 genes per cluster

In [6]:
label= 'mono'
Xtrain = data['Xall_mono']
Xtest = data['Xtest_mono']
set_clusters=[7,11,3,4,6]
n = 50
adata_c = data['Call_mono']
adata1_c = data['Ctest_mono']
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)
state = data['state']


### 2.1.1.CloudPred model

In [8]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [5])

start cloudpred
0
-----------------------------------------------------------
Iteration #1:
{'loss': 0.5924975875765085, 'accuracy': 0.7, 'soft': 0.6008832603693008, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #101:
{'loss': 0.5948702827095985, 'accuracy': 0.7, 'soft': 0.5999908357858658, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #201:
{'loss': 0.5951076559722424, 'accuracy': 0.7, 'soft': 0.5998267471790314, 'auc': 0.8, 'r2': nan}
-----------------------------------------------------------
Iteration #301:
{'loss': 0.595208403468132, 'accuracy': 0.7, 'soft': 0.5997719436883926, 'auc': 0.8, 'r2': nan}
-----------------------------------------------------------
Iteration #401:
{'loss': 0.5946061160415411, 'accuracy': 0.7, 'soft': 0.6003642529249191, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #501:
{'loss': 0.5937930531799793, 'accuracy': 

-----------------------------------------------------------
Iteration #1:
{'loss': 0.594407606124878, 'accuracy': 0.8, 'soft': 0.5662977576255799, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #101:
{'loss': 0.5948860749602318, 'accuracy': 0.8, 'soft': 0.5660720527172088, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #201:
{'loss': 0.594814969599247, 'accuracy': 0.8, 'soft': 0.566044756770134, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #301:
{'loss': 0.5947790160775185, 'accuracy': 0.8, 'soft': 0.5660418599843979, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #401:
{'loss': 0.5947425350546837, 'accuracy': 0.8, 'soft': 0.5660468101501465, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #501:
{'loss': 0.5946961849927902, 'accuracy': 0.8, 'soft': 0.566

### 2.2.Using top 20 genes per cluster

In [12]:
Xtrain = data['Xall_mono']
Xtest = data['Xtest_mono']
n = 20
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)

### 2.2.2.CloudPred class 

In [13]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [5])

start cloudpred
0
-----------------------------------------------------------
Iteration #1:
{'loss': 0.7683868914842605, 'accuracy': 0.5, 'soft': 0.49695548713207244, 'auc': 0.52, 'r2': nan}
-----------------------------------------------------------
Iteration #101:
{'loss': 0.7949044451117515, 'accuracy': 0.5, 'soft': 0.5003264814615249, 'auc': 0.52, 'r2': nan}
-----------------------------------------------------------
Iteration #201:
{'loss': 0.8052184611558915, 'accuracy': 0.6, 'soft': 0.5031049102544785, 'auc': 0.52, 'r2': nan}
-----------------------------------------------------------
Iteration #301:
{'loss': 0.8073454424738884, 'accuracy': 0.6, 'soft': 0.5047136455774307, 'auc': 0.52, 'r2': nan}
-----------------------------------------------------------
Iteration #401:
{'loss': 0.8072532325983047, 'accuracy': 0.6, 'soft': 0.5055735439062119, 'auc': 0.52, 'r2': nan}
-----------------------------------------------------------
Iteration #501:
{'loss': 0.8094344481825828, 'accurac

-----------------------------------------------------------
Iteration #1:
{'loss': 0.5052674545469756, 'accuracy': 0.8, 'soft': 0.8161316869780422, 'auc': 0.9600000000000001, 'r2': nan}
-----------------------------------------------------------
Iteration #101:
{'loss': 0.5078794023865839, 'accuracy': 0.8, 'soft': 0.8127672281116247, 'auc': 0.9600000000000001, 'r2': nan}
-----------------------------------------------------------
Iteration #201:
{'loss': 0.5126936298998658, 'accuracy': 0.8, 'soft': 0.8109817676246166, 'auc': 0.9600000000000001, 'r2': nan}
-----------------------------------------------------------
Iteration #301:
{'loss': 0.5246846019872351, 'accuracy': 0.8, 'soft': 0.802767937630415, 'auc': 0.9600000000000001, 'r2': nan}
-----------------------------------------------------------
Iteration #401:
{'loss': 0.5237997458927538, 'accuracy': 0.8, 'soft': 0.802728658169508, 'auc': 0.9600000000000001, 'r2': nan}
-----------------------------------------------------------
Iter

### 2.3.Using top 100 genes per cluster

In [15]:
Xtrain = data['Xall_mono']
Xtest = data['Xtest_mono']
n = 100
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)

### 2.3.2.GMM class 

In [16]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [5])

start cloudpred
0
-----------------------------------------------------------
Iteration #1:
{'loss': 0.23221211095001024, 'accuracy': 0.8, 'soft': 0.8504557430744171, 'auc': 1.0, 'r2': nan}
-----------------------------------------------------------
Iteration #101:
{'loss': 1.221907478120459, 'accuracy': 0.8, 'soft': 0.7723183311158209, 'auc': 0.9600000000000001, 'r2': nan}
-----------------------------------------------------------
Iteration #201:
{'loss': 3.131019627598652, 'accuracy': 0.7, 'soft': 0.6956527592039731, 'auc': 0.92, 'r2': nan}
-----------------------------------------------------------
Iteration #301:
{'loss': 2.9463641554023523, 'accuracy': 0.7, 'soft': 0.6642226375546262, 'auc': 0.92, 'r2': nan}
-----------------------------------------------------------
Iteration #401:
{'loss': 2.768774431736529, 'accuracy': 0.7, 'soft': 0.6742159569332861, 'auc': 0.92, 'r2': nan}
-----------------------------------------------------------
Iteration #501:
{'loss': 2.7856426412523434

-----------------------------------------------------------
Iteration #1:
{'loss': 0.6460537344217301, 'accuracy': 0.7, 'soft': 0.5549047246575356, 'auc': 0.8, 'r2': nan}
-----------------------------------------------------------
Iteration #101:
{'loss': 0.6456817120313645, 'accuracy': 0.7, 'soft': 0.5559014469385147, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #201:
{'loss': 0.6442605823278427, 'accuracy': 0.7, 'soft': 0.55702553242445, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #301:
{'loss': 0.6433625280857086, 'accuracy': 0.7, 'soft': 0.557744899392128, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #401:
{'loss': 0.6431437343358993, 'accuracy': 0.7, 'soft': 0.5579970583319664, 'auc': 0.76, 'r2': nan}
-----------------------------------------------------------
Iteration #501:
{'loss': 0.6427236884832382, 'accuracy': 0.7, 'soft': 0.5582

## 3.Prediction based on Monocytes+Neutrophils 

### 3.1.Using top 50 genes per cluster

In [ ]:
label='mono_neu'
Xtrain = data['Xall_mono_neu']
Xtest = data['Xtest_mono_neu']
set_clusters=[7,11,3,4,6,9,14]
n = 50
adata_c = data['Call_mono_neu']
adata1_c = data['Ctest_mono_neu']
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)
state = data['state']


### 3.1.2.GMM_class

In [ ]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [7,5])

### 3.2.Using top 20 genes per cluster

In [ ]:
n = 20
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)

### 3.2.2.GMM class 

In [ ]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [7,5])

### 3.3.Using top 100 genes per cluster

In [ ]:
n = 100
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)

### 3.3.2.GMM class 

In [ ]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [7,5])

## 4.Prediction based on Neutrophils 

### 4.1.Using top 50 genes per cluster

In [ ]:
label='neu'
Xtrain = data['Xall_neu']
Xtest = data['Xtest_neu']
set_clusters=[9,14]
n = 50
adata_c = data['Call_neu']
adata1_c = data['Ctest_neu']
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)
state = data['state']


### 4.1.2.GMM_class

In [ ]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [2])

### 4.2.Using top 20 genes per cluster

In [ ]:
n = 20
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)

### 4.2.2.GMM class 

In [ ]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [2])

### 4.3.Using top 100 genes per cluster

In [ ]:
n = 100
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)

### 4.3.2.GMM class 

In [ ]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [2])

## 5.Prediction based on All cells 

### 5.1.Using top 50 genes per cluster

In [17]:
label='all'
Xtrain = data['Xall']
Xtest = data['Xtest']
set_clusters=np.unique(markers["cluster"])
n = 50
adata_c = data['Call']
adata1_c = data['Ctest']
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)
state = data['state']


### 5.1.2.GMM_class

In [18]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [21,5])

start cloudpred
0
-----------------------------------------------------------
Iteration #1:
{'loss': 0.5392086587846279, 'accuracy': 0.8, 'soft': 0.6291739374399186, 'auc': 0.8400000000000001, 'r2': nan}
-----------------------------------------------------------
Iteration #101:
{'loss': 0.5444179028272629, 'accuracy': 0.8, 'soft': 0.6299974843859673, 'auc': 0.8400000000000001, 'r2': nan}
-----------------------------------------------------------
Iteration #201:
{'loss': 0.5471087500452996, 'accuracy': 0.9, 'soft': 0.6301737666130066, 'auc': 0.8400000000000001, 'r2': nan}
-----------------------------------------------------------
Iteration #301:
{'loss': 0.5483154460787774, 'accuracy': 0.9, 'soft': 0.6302132487297059, 'auc': 0.8400000000000001, 'r2': nan}
-----------------------------------------------------------
Iteration #401:
{'loss': 0.5488010525703431, 'accuracy': 0.9, 'soft': 0.63025411516428, 'auc': 0.8400000000000001, 'r2': nan}
----------------------------------------------

-----------------------------------------------------------
Iteration #601:
{'loss': 0.4831590569570835, 'accuracy': 0.7, 'soft': 0.7246729969978333, 'auc': 0.7916666666666667, 'r2': nan}
-----------------------------------------------------------
Iteration #701:
{'loss': 0.4830219260074955, 'accuracy': 0.7, 'soft': 0.7246902897953987, 'auc': 0.7916666666666667, 'r2': nan}
-----------------------------------------------------------
Iteration #801:
{'loss': 0.4827943483251147, 'accuracy': 0.7, 'soft': 0.7247406922280788, 'auc': 0.7916666666666667, 'r2': nan}
-----------------------------------------------------------
Iteration #901:
{'loss': 0.482524744803959, 'accuracy': 0.7, 'soft': 0.7247906468808651, 'auc': 0.7916666666666667, 'r2': nan}
**********************************************
{'loss': 0.48228604431205896, 'accuracy': 0.7, 'soft': 0.724799694865942, 'auc': 0.7916666666666667, 'r2': nan}
best score for center0.48228604431205896_5
{'loss': 0.48228604431205896, 'accuracy': 0.7, 

KeyboardInterrupt: 

### 5.2.Using top 20 genes per cluster

In [ ]:
Xtrain = data['Xall']
Xtest = data['Xtest']
n = 20
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)

### 5.2.2.GMM class 

In [ ]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [21,5])

### 5.3.Using top 100 genes per cluster

In [ ]:
Xtrain = data['Xall']
Xtest = data['Xtest']
n = 100
Xtrain,Xtest = get_data(Xtrain,Xtest,set_clusters,n,adata_c,adata1_c,markers)

### 5.3.2.GMM class 

In [ ]:
for seed in [0,42,10,1234,4321]:
    cloud_pred(Xtrain, Xtest, seed, [21,5])